In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import json
import zipfile

# 1️⃣ DATASET EXTRACTION
zip_path = "/content/drive/MyDrive/ml_pr.zip"
extract_path = "/content/dataset2"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted successfully!")

dataset_folder = os.path.join(extract_path, "food_ml")
train_dir = os.path.join(dataset_folder, 'training')
val_dir = os.path.join(dataset_folder, 'validation')

if not os.path.exists(train_dir) or not os.path.exists(val_dir):
    raise FileNotFoundError("Dataset folder structure is incorrect.")

print("📂 Train Directory:", train_dir)
print("📂 Validation Directory:", val_dir)

# 2️⃣ DATA AUGMENTATION
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,  # 🔥 Increased batch size
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=64,  # 🔥 Increased batch size
    class_mode='categorical'
)

# 3️⃣ MODEL ARCHITECTURE (With Fine-Tuning)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # 🔥 Enable fine-tuning

# Unfreeze last few layers for better accuracy
for layer in base_model.layers[:-20]:  # 🔥 Unfreeze last 20 layers
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(11, activation='softmax')(x)  # 11 Classes

model = Model(inputs=base_model.input, outputs=x)

# 4️⃣ COMPILATION WITH IMPROVED LEARNING RATE
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 5️⃣ CALLBACKS
checkpoint = ModelCheckpoint('resnet50_best_model.h5', monitor='val_accuracy', save_best_only=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 6️⃣ MODEL TRAINING
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=40,  # 🔥 Increased epochs
    callbacks=[checkpoint, lr_scheduler, early_stopping]
)

# 7️⃣ MODEL SAVING
model.save('resnet50_final_model.h5')

# 8️⃣ SAVE CLASS INDICES
with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)

# 9️⃣ IMAGE PREDICTION FUNCTION
def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    return predicted_class, prediction


✅ Dataset extracted successfully!
📂 Train Directory: /content/dataset2/food_ml/training
📂 Validation Directory: /content/dataset2/food_ml/validation
Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1736 - loss: 2.3163

155/155 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.1738 - loss: 2.3159 - val_accuracy: 0.1577 - val_loss: 2.2823 - learning_rate: 1.0000e-04
Epoch 2/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 948ms/step - accuracy: 0.2434 - loss: 2.1403

155/155 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.2434 - loss: 2.1403 - val_accuracy: 0.2545 - val_loss: 2.1300 - learning_rate: 1.0000e-04
Epoch 3/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 921ms/step - accuracy: 0.2635 - loss: 2.0891

...

155/155 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.3948 - loss: 1.7546 - val_accuracy: 0.4327 - val_loss: 1.6497 - learning_rate: 9.0000e-06
Epoch 19/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.3904 - loss: 1.7754 - val_accuracy: 0.4227 - val_loss: 1.6989 - learning_rate: 9.0000e-06
Epoch 20/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.3804 - loss: 1.7662 - val_accuracy: 0.4044 - val_loss: 1.7426 - learning_rate: 9.0000e-06
Epoch 21/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 920ms/step - accuracy: 0.3937 - loss: 1.7617


Epoch 21: ReduceLROnPlateau reducing learning rate to 2.6999998226528985e-06.
155/155 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.3936 - loss: 1.7617 - val_accuracy: 0.4335 - val_loss: 1.6544 - learning_rate: 9.0000e-06
Epoch 22/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.3843 - loss: 1.7437 - val_accuracy: 0.4286 - val_loss: 1.6573 - learning_rate: 2.7000e-06
Epoch 23/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.3874 - loss: 1.7481 - val_accuracy: 0.4277 - val_loss: 1.6494 - learning_rate: 2.7000e-06
Epoch 24/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.3878 - loss: 1.7581 - val_accuracy: 0.4315 - val_loss: 1.6541 - learning_rate: 2.7000e-06
Epoch 25/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 907ms/step - accuracy: 0.3895 - loss: 1.7506

155/155 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.3895 - loss: 1.7506 - val_accuracy: 0.4353 - val_loss: 1.6396 - learning_rate: 2.7000e-06
Epoch 26/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.3909 - loss: 1.7450 - val_accuracy: 0.4327 - val_loss: 1.6550 - learning_rate: 2.7000e-06
Epoch 27/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.3954 - loss: 1.7483 - val_accuracy: 0.4341 - val_loss: 1.6445 - learning_rate: 2.7000e-06
Epoch 28/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 930ms/step - accuracy: 0.3921 - loss: 1.7320
Epoch 28: ReduceLROnPlateau reducing learning rate to 8.099999604382901e-07.
155/155 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - accuracy: 0.3921 - loss: 1.7320 - val_accuracy: 0.4300 - val_loss: 1.6530 - learning_rate: 2.7000e-06
Epoch 29/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.3926 - loss: 1.7432 - val_accuracy: 0.4353 - val_loss: 1.6368 - learning_rate: 8.1000e-07
Epoch 30/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 914ms/step - accuracy: 0

155/155 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.3970 - loss: 1.7349 - val_accuracy: 0.4367 - val_loss: 1.6375 - learning_rate: 8.1000e-07
Epoch 31/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.4036 - loss: 1.7098 - val_accuracy: 0.4362 - val_loss: 1.6350 - learning_rate: 8.1000e-07
Epoch 32/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.3957 - loss: 1.7363 - val_accuracy: 0.4367 - val_loss: 1.6329 - learning_rate: 8.1000e-07
Epoch 33/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.4035 - loss: 1.7291 - val_accuracy: 0.4367 - val_loss: 1.6372 - learning_rate: 8.1000e-07
Epoch 34/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 920ms/step - accuracy: 0.3851 - loss: 1.7507

155/155 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.3851 - loss: 1.7506 - val_accuracy: 0.4373 - val_loss: 1.6365 - learning_rate: 8.1000e-07
Epoch 35/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 917ms/step - accuracy: 0.4026 - loss: 1.7281
Epoch 35: ReduceLROnPlateau reducing learning rate to 2.4299998813148704e-07.
155/155 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.4026 - loss: 1.7280 - val_accuracy: 0.4359 - val_loss: 1.6410 - learning_rate: 8.1000e-07
Epoch 36/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.3939 - loss: 1.7263 - val_accuracy: 0.4362 - val_loss: 1.6379 - learning_rate: 2.4300e-07
Epoch 37/40
155/155 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.3877 - loss: 1.7377 - val_accuracy: 0.4364 - val_loss: 1.6346 - learning_rate: 2.4300e-07


In [3]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Sample image path
img_path = "/content/Baked-French-Fries-Recipe-1200.jpg"  # Change this to actual image path
img = image.load_img(img_path, target_size=(224, 224))  # Resize to model input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Expand dims for batch format
img_array /= 255.0  # Normalize

# Predict
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)  # Get highest probability class
print(f"Predicted Class Index: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted Class Index: 10
